# Tutorial Básico #EX03
# Multiclass Classification Of Flower Species

# Clasificación Multiclase de Especies Florales

En este tutorial de proyecto, descubrirá cómo puede usar Keras para desarrollar y evaluar modelos de redes neuronales para problemas de clasificación de clases múltiples.


# Conjunto de datos de clasificación de flores de iris

En este tutorial utilizaremos el problema de aprendizaje automático estándar denominado conjunto de datos de flores iris. Este conjunto de datos está bien estudiado y es un buen problema para practicar en redes neuronales porque las 4 variables de entrada son numéricas y tienen la misma escala en centímetros. Cada instancia describe las propiedades de una medición.
Las mediciones de flujo y la variable de salida son especies específicas de iris. Los atributos para este conjunto de datos se pueden resumir de la siguiente manera:

1. Longitud del sépalo en centímetros.
2. Ancho sepal en centímetros.
3. Longitud del pétalo en centímetros.
4. Ancho del pétalo en centímetros.
5. Clase.

Este es un problema de clasificación multiclase, lo que significa que hay más de dos clases para predecir, de hecho, hay tres especies de flores. Este es un tipo importante de problema para practicar con redes neuronales porque los tres valores de clase requieren un manejo especializado. A continuación se muestra una muestra de las primeras cinco de las 150 instancias:

5.1,3.5,1.4,0.2,Iris-setosa

4.9,3.0,1.4,0.2,Iris-setosa

4.7,3.2,1.3,0.2,Iris-setosa

4.6,3.1,1.5,0.2,Iris-setosa

5.0,3.6,1.4,0.2,Iris-setosa


El conjunto de datos de la flor del iris es un problema bien estudiado y podemos esperar una precisión del modelo en el rango del 95% al 97%. Esto proporciona un buen objetivo para obtener durante el desarrollo de nuestros modelos. Puede descargar el conjunto de datos de flores del iris desde el repositorio de Aprendizaje automático de UCI y ubicarlo en su directorio de trabajo actual con el nombre de archivo iris.csv.


# Importar Clases y Funciones

Podemos comenzar importando todas las clases y funciones que necesitaremos en este tutorial. Esto incluye tanto la funcionalidad que requerimos de Keras, como la carga de datos desde Pandas, así como la preparación de datos y la evaluación del modelo desde scikit-learn.

Además, necesitamos inicializar el generador de números aleatorios a un valor constante. Esto es importante para garantizar que los resultados que obtenemos de este modelo se puedan lograr nuevamente de manera precisa. Además, asegura que el proceso estocástico de entrenamiento de un modelo de red neuronal pueda reproducirse.

In [ ]:
# Multiclass Classification with the Iris Flowers Dataset
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# Cargar el conjunto de datos

El conjunto de datos se puede cargar directamente. Debido a que la variable de salida contiene cadenas (strings), es más fácil cargar los datos utilizando pandas. Luego podemos dividir los atributos (columnas) en variables de entrada (X) y variables de salida (Y).

In [ ]:
# load dataset
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]
print(X)

# Codificar la variable de salida

La variable de salida contiene tres valores de cadena diferentes. Al modelar problemas de clasificación multiclase usando redes neuronales, es una buena práctica remodelar el atributo de salida de un vector que contiene valores para cada valor de clase para que sea una matriz con un valor booleano para cada valor de clase y si una instancia dada tiene esa clase o no. Esto se denomina codificación en caliente (one hot encoding) o creación de variables ficticias a partir de una variable categórica. Por ejemplo, en este problema, los tres valores de clase son Iris-setosa, Iris-versicolor e Iris-virginica. Si tuviéramos las tres observaciones:

Iris-setosa

Iris-versicolor

Iris-virginica

Podemos convertir esto en una matriz binaria codificada para cada instancia de datos que se vería de la siguiente manera:

Iris-setosa, Iris-versicolor, Iris-virginica

1,                  0,              0

0,                  1,              0

0,                  0,              1

Podemos hacer esto mediante la primera codificación de las cadenas de forma coherente a enteros utilizando la clase LabelEncoder de scikit-learn. Luego convierta el vector de enteros en una codificación one hot encoding utilizando la función Keras to_categorical().


In [ ]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
# print(dummy_y)

# Definir el modelo de red neuronal

La biblioteca de Keras proporciona clases para permitirle usar modelos de redes neuronales desarrollados con Keras en scikit-learn como vimos en el ejemplo (clase) anterior. Hay una clase KerasClassifier en Keras que se puede usar como un Estimador en scikit-learn, el tipo base de modelo en la biblioteca. El KerasClassifier toma el nombre de una función como un argumento. Esta función debe devolver el modelo de red neuronal construido, listo para el entrenamiento.

A continuación se muestra una función que creará una red neuronal base para el problema de clasificación de iris. Crea una red simple completamente conectada con una capa oculta que contiene 4 neuronas, el mismo número de entradas (podría ser cualquier número de neuronas). La capa oculta utiliza una función de activación de rectificador lo cual es una buena práctica. Debido a que usamos una codificación one hot para nuestro conjunto de datos de iris, la capa de salida debe crear 3 valores de salida, uno para cada clase. El valor de salida con el valor más grande se tomará como la clase predicha por el modelo. La topología de red de esta simple red neuronal de una capa se puede resumir como:

4 inputs -> [4 hidden nodes] -> 3 outputs

Tenga en cuenta que usamos una función de activación sigmoide en la capa de salida. Esto es para asegurar que los valores de salida estén en el rango de 0 y 1 y se puedan usar como probabilidades pronosticadas. Finalmente, la red utiliza el eficiente algoritmo de optimización de descenso por gradiente de ADAM con una función de pérdida logarítmica, que se denomina crossentropy categórica en Keras.

In [ ]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=4, init='normal', activation='relu'))
    model.add(Dense(3, init='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Estimador

Ahora podemos crear nuestro KerasClassifier para su uso en scikit-learn. También podemos pasar argumentos en la construcción de la clase KerasClassifier que se pasará a la función fit() internamente utilizada para entrenar la red neuronal. Aquí, pasamos el número de épocas como 200 y tamaño de lote como 5 para usar al entrenar el modelo. La depuración también se desactiva cuando se entrena estableciendo verbose en 0.

In [ ]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5)

# Evaluar el modelo con la validación cruzada k-Fold

Ahora podemos evaluar el modelo de red neuronal en nuestros datos de entrenamiento. La biblioteca scikit-learn tiene una excelente capacidad para evaluar modelos utilizando un conjunto de técnicas. El estándar de oro para evaluar modelos de aprendizaje automático es la validación cruzada k-fold. Primero podemos definir el procedimiento de evaluación del modelo. Aquí, establecemos que el número de carpetas sea 10 (un valor predeterminado excelente) y que mezclen los datos antes de particionarlos.

Luego podemos evaluar nuestro modelo (estimador) en nuestro conjunto de datos (X y dummy_y) utilizando el procedimiento de validación cruzada de 10 veces (kfold). La evaluación del modelo solo toma algunos minutos y devuelve un objeto que describe la evaluación de los 10 modelos construidos para cada una de las divisiones del conjunto de datos.

In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


# Código completo 


In [ ]:
# Multiclass Classification with the Iris Flowers Dataset
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load dataset
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]
#print(X)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
# print(dummy_y)

# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=4, init='normal', activation='relu'))
    model.add(Dense(3, init='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

# Evaluar el modelo directamente

Ejemplo de como generar una red neuronal que clasifique multiples clases directamente, sin validación cruzada. 

In [1]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
# define baseline model

# create model
model = Sequential()
model.add(Dense(4, input_dim=4, init='normal', activation='relu'))
model.add(Dense(3, init='normal', activation='sigmoid'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X, dummy_y, nb_epoch=200, batch_size=5)  

# evaluate the model
scores = model.evaluate(X,  dummy_y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, input_dim=4, activation="relu", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(3, activation="sigmoid", kernel_initializer="normal")`
C:\ProgramData\Anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/200
150/150 [==============================] - 2s 16ms/step - loss: 1.0995 - acc: 0.2067
Epoch 2/200
150/150 [==============================] - 0s 1ms/step - loss: 1.0962 - acc: 0.3600
Epoch 3/200
150/150 [==============================] - 0s 1ms/step - loss: 1.0916 - acc: 0.3333
Epoch 4/200
150/150 [==============================] - 0s 1ms/step - loss: 1.0845 - acc: 0.3333
Epoch 5/200
150/150 [==============================] - 0s 1ms/step - loss: 1.0732 - acc: 0.3533
Epoch 6/200
150/150 [==============================] - 0s 1ms/step - loss: 1.0570 - acc: 0.6333
Epoch 7/200
150/150 [==============================] - 0s 1ms/step - loss: 1.0366 - acc: 0.6667
Epoch 8/200
150/150 [==============================] - 0s 1ms/step - loss: 1.0129 - acc: 0.6667
Epoch 9/200
150/150 [==============================] - 0s 1ms/step - loss: 0.9857 - acc: 0.6667
Epoch 10/200
150/150 [==============================] - 0s 1ms/step - loss: 0.9559 - acc: 0.6667
Epoch 11/200
150/150 [================

150/150 [==============================] - 0s 884us/step - loss: 0.1163 - acc: 0.9667
Epoch 169/200
150/150 [==============================] - 0s 1ms/step - loss: 0.1173 - acc: 0.9733
Epoch 170/200
150/150 [==============================] - 0s 1ms/step - loss: 0.1168 - acc: 0.9667
Epoch 171/200
150/150 [==============================] - 0s 997us/step - loss: 0.1158 - acc: 0.9667
Epoch 172/200
150/150 [==============================] - 0s 971us/step - loss: 0.1159 - acc: 0.9667
Epoch 173/200
150/150 [==============================] - 0s 1ms/step - loss: 0.1145 - acc: 0.9733
Epoch 174/200
150/150 [==============================] - 0s 977us/step - loss: 0.1141 - acc: 0.9667
Epoch 175/200
150/150 [==============================] - 0s 1ms/step - loss: 0.1146 - acc: 0.9667
Epoch 176/200
150/150 [==============================] - 0s 1ms/step - loss: 0.1178 - acc: 0.9800
Epoch 177/200
150/150 [==============================] - 0s 1ms/step - loss: 0.1136 - acc: 0.9733
Epoch 178/200
150/150 [===

# Revisión de predicción


In [2]:
import numpy as np
model.predict(np.reshape(X[100], [1, -1]))

array([[7.19515116e-12, 4.67439677e-04, 1.14623785e-01]], dtype=float32)

In [3]:
dummy_y[100]

array([0., 0., 1.])